# 2a2 Comparing Curvatures on Toy Manifolds

::: {#cell-1 .cell 0=‘h’ 1=‘i’ 2=‘d’ 3=‘e’ execution_count=66}

``` python
## Standard libraries
import os
import math
import numpy as np
import time
from fastcore.all import *
from nbdev.showdoc import *

# Configure environment
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false' # Tells Jax not to hog all of the memory to this process.

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.auto import tqdm, trange

## project specifics
import diffusion_curvature
import pygsp
import jax
import jax.numpy as jnp
jax.devices()

from diffusion_curvature.graphs import *
from diffusion_curvature.datasets import *
from diffusion_curvature.core import *
from diffusion_curvature.utils import *
from diffusion_curvature.comparison_space import *

%load_ext autoreload
%autoreload 2
```

    The autoreload extension is already loaded. To reload it, use:
      %reload_ext autoreload

:::

> Which will emerge victorious?

The comparison consists of two parts: 1. A figure showing the visual
differences, both in 3d scatter plots, and correlation plots. 2. Tables
computing the resilience to noise across methods.

# Machinery

Load up the results:

In [67]:
curvatures = {
    'Ground Truth Curvature' : np.load("../../results/toy_data_saved_1000_more_noise_curvatures.npz"),
    'Diffusion Curvature' : np.load("../../results/toy_dataset_entropic_fixed_subtraction.npz"),
    # 'Unsigned Diffusion Curvature' : np.load("../../results/toy_dataset_neurips_laziness_no_comparison.npz"),
    # 'AdaL-PCA' : np.load("../../results/toy_dataset_toy_datasets_principal_curvature_heuristic.npz"),
    # 'AdaL-PCA' : np.load("../../results/toy_dataset_toy_datasets_principal_curvature_heuristic_filtered.npz"), # outlier points filtered out
    # 'AdaL-PCA' : np.load("../../results/toy_dataset_toy_datasets_principal_curvature_no_heuristic_08_15.npz"),
    # 'Unsigned Diffusion Curvature' : np.load("../../results/toy_dataset_neurips_laziness_no_comparison.npz"),
    'Hickok & Blumberg' : np.load("../../results/toy_dataset_toy_datasets_hickock_curvature_r3.14.npz"),
    'Ollivier Ricci Curvature' : np.load("../../results/toy_dataset_toy_datasets_ollivier_ricci_knn_50.npz"),
    'Forman Ricci Curvature' : np.load('../../results/toy_dataset_toy_datasets_forman_ricci_k_50.npz'),
}
scaled_curvatures = dict(curvatures)

And the data

In [68]:
# Establish Datasets
load_from_saved = True
filename = "../../results/toy_data_saved_1000_more_noise"
if load_from_saved:
    Xs = np.load(filename + '.npz')
    Ks = np.load(filename + '_curvatures.npz')
else:
    n_points = 1000
    Xs = {}
    Ks = {}
    Xs['Torus'], Ks['Torus'] = torus(n_points, use_guide_points=False, seed=19323)
    Xs['plane'] = np.hstack([plane(n_points, dim=2),np.zeros((n_points,1))])
    Ks['plane'] = np.zeros(len(Xs['plane']))
    Xs['Saddle'], Ks['Saddle'] = paraboloid(n_points, use_guide_points=False, seed=19323)
    Xs['Sphere'], Ks['Sphere'] = sphere(n_points,)
    Xs['Ellipsoid'], Ks['Ellipsoid'] = ellipsoid(n_points, seed=19323)
    Xs['Hyperboloid'], Ks['Hyperboloid'] = hyperboloid(n_points, seed=19323)
    # Noised Versions
    for noise_scale in [0.1, 0.2, 0.3, 0.4, 0.5]: # [0.05, 0.1, 0.15, 0.2, 0.25]: 
        Xs[f'Torus noise = {noise_scale}'], Ks[f'Torus noise = {noise_scale}'] = torus(n_points, use_guide_points=False, seed=19323, noise = noise_scale)
        # Xs['plane noise = {noise_scale}'] = np.hstack([plane(n_points, dim=2),np.zeros((n_points,1))])
        # Ks['plane noise = {noise_scale}'] = np.zeros(len(Xs['plane']))
        Xs[f'Saddle noise = {noise_scale}'], Ks[f'Saddle noise = {noise_scale}'] = paraboloid(n_points, use_guide_points=False, seed=19323, noise = noise_scale)
        # Xs['sphere = {noise_scale}'], Ks['sphere = {noise_scale}'] = sphere(n_points,)
        Xs[f'Ellipsoid noise = {noise_scale}'], Ks[f'Ellipsoid noise = {noise_scale}'] = ellipsoid(n_points, seed=19323, noise = noise_scale)
        Xs[f'hyperboloid noise = {noise_scale}'], Ks[f'hyperboloid noise = {noise_scale}'] = hyperboloid(n_points, seed=19323, noise = noise_scale)
    np.savez(filename + '.npz', **Xs)
    np.savez(filename + '_curvatures.npz', **Ks)

In [69]:
Xs['Ellipsoid'].shape

(1000, 3)

In [70]:
# # Establish Datasets
# n_points = 2000
# Xs = {}
# Ks = {}
# Xs['Torus'], Ks['Torus'] = torus(n_points, use_guide_points=False, seed=19323)
# Xs['Saddle'], Ks['Saddle'] = paraboloid(n_points, use_guide_points=False, seed=19323)
# Xs['sphere'], Ks['sphere'] = sphere(n_points,)
# Xs['Ellipsoid'], Ks['Ellipsoid'] = ellipsoid(n_points, seed=19323)
# Xs['hyperboloid'], Ks['hyperboloid'] = hyperboloid(n_points, seed=19323)
# # Noised Versions
# for noise_scale in [0.1, 0.2, 0.3, 0.4, 0.5]:
#     Xs[f'Torus noise = {noise_scale}'], Ks[f'Torus noise = {noise_scale}'] = torus(n_points, use_guide_points=False, seed=19323, noise = noise_scale)
#     # Xs['plane noise = {noise_scale}'] = np.hstack([plane(n_points, dim=2),np.zeros((n_points,1))])
#     # Ks['plane noise = {noise_scale}'] = np.zeros(len(Xs['plane']))
#     Xs[f'Saddle noise = {noise_scale}'], Ks[f'Saddle noise = {noise_scale}'] = paraboloid(n_points, use_guide_points=False, seed=19323, noise = noise_scale)
#     # Xs['sphere = {noise_scale}'], Ks['sphere = {noise_scale}'] = sphere(n_points,)
#     Xs[f'Ellipsoid noise = {noise_scale}'], Ks[f'Ellipsoid noise = {noise_scale}'] = ellipsoid(n_points, seed=19323, noise = noise_scale)
#     Xs[f'hyperboloid noise = {noise_scale}'], Ks[f'hyperboloid noise = {noise_scale}'] = hyperboloid(n_points, seed=19323, noise = noise_scale)

In [71]:
curvatures['Hickok & Blumberg']['Torus'].shape

(1000,)

# Results

In [72]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import colors
import numpy as np

# Assuming Xs, labels, ground_truth_labels, and key_names are defined
# For example:
key_names = ['Torus', 'Saddle', 'Ellipsoid']
labels = curvatures
ground_truth_labels = Ks

# Set the style of seaborn
sns.set(style="whitegrid")

fig_shape = (len(curvatures.keys()) + 1, len(key_names))
method_colors = ['green', 'olive', 'navy', 'orange', 'purple']
assert len(method_colors) == len(curvatures.keys())

ordered_keys = curvatures.keys() #['Gaussian', 'Adal-PCA', 'Unsigned Diffusion Curvature', 'Hickok & Blumberg']
assert set(ordered_keys) == set(curvatures.keys())

# Create a figure and axes with 2 rows and 3 columns
fig, axs = plt.subplots(fig_shape[0], fig_shape[1], figsize=(fig_shape[0]*2, fig_shape[1]*5), subplot_kw={'projection': '3d'} if plt.matplotlib.__version__ < '3.4.0' else None)

for i, key in enumerate(key_names):
    # 3D scatter plot for the first row
    lim = np.max(np.linalg.norm(Xs[key], axis=1))
    # if plt.matplotlib.__version__ >= '3.4.0':

    # plot the curvature for each method
    for j, method_name in enumerate(ordered_keys):
        axs[j, i].remove()
        axs[j, i] = fig.add_subplot(fig_shape[0], fig_shape[1], i + j*fig_shape[1] + 1, projection='3d')
        axs[j, i].axes.set_xlim3d(left=-lim, right=lim)
        axs[j, i].axes.set_ylim3d(bottom=-lim, top=lim)
        axs[j, i].axes.set_zlim3d(bottom=-lim, top=lim)
        axs[j, i].set_xticks([])
        axs[j, i].set_yticks([])
        axs[j, i].set_zticks([])

        scaled_labels = labels[method_name][key]
        scaled_labels = scaled_labels / (max(scaled_labels) - min(scaled_labels))
        scatter3d = axs[j, i].scatter(Xs[key][:, 0], Xs[key][:, 1], Xs[key][:, 2], c=scaled_labels, cmap = 'coolwarm')
        if i == 1:
            axs[j, i].set_title(f'{method_name}',fontsize=15, c = method_colors[j])

    # Scatter plot for the second row, colored by labels
    # axs[j+1, i].remove()
    # axs[j+1, i] = fig.add_subplot(fig_shape[0], fig_shape[1], i+4)
    for k, method_name in enumerate(ordered_keys):
        # min max scaling
        scaled_labels = labels[method_name][key]
        scaled_labels = scaled_labels / (max(scaled_labels) - min(scaled_labels))
        scatter2d = axs[j+1, i].scatter(ground_truth_labels[key], scaled_labels, c = method_colors[k], s = 1)
        axs[j+1,i].set_aspect('equal')
    # axs[1, i].set_title(f'')
    axs[j+1, i].set_xlabel('Gaussian Curvature')
    axs[j+1, i].set_ylabel('Estimated Curvature')
    
    
    # Highlight the origin lines in 2D scatter plots
    axs[j+1, i].axhline(0, color='gray', linewidth=1, linestyle='--')
    axs[j+1, i].axvline(0, color='gray', linewidth=1, linestyle='--')

    # # Add colorbar for each subplot
    # fig.colorbar(scatter3d, ax=axs[0, i], fraction=0.046, pad=0.04)
    # fig.colorbar(scatter2d, ax=axs[1, i], fraction=0.046, pad=0.04)
plt.suptitle("Estimated Gaussian Curvatures of 2-Manifolds", fontsize=30)
plt.tight_layout()
plt.show()


# Table

In [73]:
import pandas as pd

In [74]:
from sklearn.metrics import mean_squared_error
def root_mse_between(labels, method, dataset):
    truth = labels['Ground Truth Curvature'][dataset]
    computed = labels[method][dataset]
    return np.sqrt(mean_squared_error(truth, computed))

In [75]:
from scipy.stats import energy_distance
def energy_between(labels, method, dataset):
    truth = labels['Ground Truth Curvature'][dataset]
    computed = labels[method][dataset]
    return energy_distance(truth, computed)

In [76]:
from scipy.stats import pearsonr
def pearson_correlation_between(labels, method, dataset):
    truth = labels['Ground Truth Curvature'][dataset]
    computed = labels[method][dataset]
    rho, confidence = pearsonr(truth, computed)
    return rho

In [79]:
noise_scales = [0.1, 0.2, 0.3, 0.4, 0.5]
dataset_names = ['Torus'] + [f'Torus noise = {x}' for x in noise_scales] + ['Saddle'] + [f'Saddle noise = {x}' for x in noise_scales] + ['Ellipsoid'] + [f'Ellipsoid noise = {x}' for x in noise_scales]

metrics_collected = {
        'RMSE':{}, 'ED':{}, 'PCC':{}
    }

for method in labels.keys():
    if method == 'Ground Truth Curvature': 
        pass
    else:
        print(method)
        # metrics_collected['RMSE'][method] = {}
        metrics_collected['ED'][method] = {}
        metrics_collected['PCC'][method] = {}
        
        for dname in dataset_names:
            try:
                # metrics_collected['RMSE'][method][dname] = root_mse_between(curvatures, method, dname)
                metrics_collected['ED'][method][dname] = energy_between(curvatures, method, dname)
                metrics_collected['PCC'][method][dname] = pearson_correlation_between(curvatures, method, dname)
            except KeyError:
                print(f"no such dname {dname} for {method}")
                pass

Diffusion Curvature
Hickok & Blumberg
Ollivier Ricci Curvature
Forman Ricci Curvature

In [80]:
root_mse_between(curvatures, 'Hickok & Blumberg', 'Torus')

1.3590765614253006

In [81]:
metrics_collected['RMSE']

{}

In [82]:
metrics_collected['ED']

{'Diffusion Curvature': {'Torus': 0.7740658266594376,
  'Torus noise = 0.1': 0.867022809002684,
  'Torus noise = 0.2': 0.9376026047242412,
  'Torus noise = 0.3': 1.0657236095135834,
  'Torus noise = 0.4': 1.0286043337856654,
  'Torus noise = 0.5': 1.0802844806070007,
  'Saddle': 0.8288120709948275,
  'Saddle noise = 0.1': 0.5169964473837296,
  'Saddle noise = 0.2': 0.23448069055934675,
  'Saddle noise = 0.3': 0.3549692062446602,
  'Saddle noise = 0.4': 0.4502585670831864,
  'Saddle noise = 0.5': 0.560606218749792,
  'Ellipsoid': 0.3310375924823877,
  'Ellipsoid noise = 0.1': 0.6674219361129565,
  'Ellipsoid noise = 0.2': 1.1164256100964751,
  'Ellipsoid noise = 0.3': 1.1355828892798994,
  'Ellipsoid noise = 0.4': 1.2604351597491663,
  'Ellipsoid noise = 0.5': 1.2754519155194124},
 'Hickok & Blumberg': {'Torus': 0.9612837972966679,
  'Torus noise = 0.1': 0.556147994862657,
  'Torus noise = 0.2': 0.3528856750399482,
  'Torus noise = 0.3': 0.44427292445851274,
  'Torus noise = 0.4': 0.410

In [83]:
metrics_collected['PCC']

{'Diffusion Curvature': {'Torus': 0.9211373039466606,
  'Torus noise = 0.1': 0.9508405158083564,
  'Torus noise = 0.2': 0.9584977988369827,
  'Torus noise = 0.3': 0.9227958011625403,
  'Torus noise = 0.4': 0.8619257583577243,
  'Torus noise = 0.5': 0.7918940147392998,
  'Saddle': 0.753273282456445,
  'Saddle noise = 0.1': 0.7664653550440301,
  'Saddle noise = 0.2': 0.6730828551904157,
  'Saddle noise = 0.3': 0.5362164343788511,
  'Saddle noise = 0.4': 0.4256353544116588,
  'Saddle noise = 0.5': 0.33854201910899373,
  'Ellipsoid': 0.6248415260956578,
  'Ellipsoid noise = 0.1': 0.588868421119645,
  'Ellipsoid noise = 0.2': 0.6139538178140781,
  'Ellipsoid noise = 0.3': 0.6173478749634128,
  'Ellipsoid noise = 0.4': 0.5688907443222231,
  'Ellipsoid noise = 0.5': 0.5205119497849594},
 'Hickok & Blumberg': {'Torus': 0.9541433506262154,
  'Torus noise = 0.1': 0.9450637731080545,
  'Torus noise = 0.2': 0.8555159097148427,
  'Torus noise = 0.3': 0.7373888515237562,
  'Torus noise = 0.4': 0.657

In [85]:


# # Define the LaTeX table structure
# def create_latex_table(metrics):
#     header = r"""\begin{tabular}{lc|cccccc|cccccc|cccccc}
# \toprule
# \multirow{2}{*}{\textbf{Model}} & \multirow{2}{*}{\textbf{Metric}} & \multicolumn{6}{c|}{\textbf{Torus}} & \multicolumn{6}{c|}{\textbf{Ellipsoid}} & \multicolumn{6}{c}{\textbf{Saddle}} \\
# & & 0.0 & 0.05 & 0.10 & 0.15 & 0.20 & \multicolumn{1}{c|}{0.25} & 0.0 & 0.05 & 0.10 & 0.15 & 0.20 & \multicolumn{1}{c|}{0.25} & 0.0 & 0.05 & 0.10 & 0.15 & 0.20 & 0.25 \\
# \midrule
# """
#     footer = r"\bottomrule\n\end{tabular}"

#     # Start building the table content
#     table_content = header

#     # Function to find the best values for bolding
#     def find_best_values(data, surfaces, noises, metric):
#         best_values = {}
#         for surface in surfaces:
#             for noise in noises:
#                 key = f"{surface} noise = {noise}" if noise > 0 else surface
#                 values = [model_data.get(key, float('-inf') if metric == 'PCC' else float('inf')) for model_data in data.values()]
#                 best_value = max(values) if metric == 'PCC' else min(values)
#                 best_values[(surface, noise)] = best_value
#         return best_values

#     # Iterate over each metric
#     for metric, models in metrics.items():
#         best_values = find_best_values(models, ["Torus", "Ellipsoid", "Saddle"], [0, 0.05, 0.1, 0.15, 0.2, 0.25], metric)
#         # Iterate over each model
#         for model, data in models.items():
#             model_escaped = model.replace('&', r'\&')  # Escape '&' in model names
#             table_content += f"{model_escaped} & {metric} "
#             for surface in ["Torus", "Ellipsoid", "Saddle"]:
#                 for noise in [0, 0.05, 0.1, 0.15, 0.2, 0.25]:
#                     key = f"{surface} noise = {noise}" if noise > 0 else surface
#                     value = data.get(key, "")
#                     if value:
#                         # Check if the value is the best and should be bolded
#                         if value == best_values[(surface, noise)]:
#                             table_content += f"& \\textbf{{{value:.3f}}} "
#                         else:
#                             table_content += f"& {value:.3f} "
#                     else:
#                         table_content += "& - "
#             table_content += r"\\"
#             table_content += "\n"
#         table_content += r"\midrule" + "\n"

#     table_content += footer
#     return table_content

# # Generate the LaTeX table
# latex_table = create_latex_table(metrics_collected)

# # Print the LaTeX table
# print(latex_table)


In [86]:
# Flatten the nested dictionary
data = []
for metric, models in metrics_collected.items():
    for model, results in models.items():
        row = {'Model': model, 'Metric': metric}
        row.update(results)
        data.append(row)

# Create a DataFrame
metric_df = pd.DataFrame(data)

In [88]:
from great_tables import GT, style, loc, exibble
from great_tables.data import sp500
import pandas as pd

df = metric_df

# Group the data by metric for proper sorting and display
df = df.sort_values(by=['Metric', 'Model']).reset_index(drop=True)

# Create the table with row groups
table = (
    GT(df, rowname_col="Model", groupname_col="Metric")
    .tab_header(title="Curvature under Noise", subtitle="2-Manifolds with Noise")
    .tab_stubhead(label="Model & Metric")
    .cols_label(
        Model="Model",
        Metric="Metric",
        **{col: col for col in df.columns if col not in ["Model", "Metric"]}
    )
    .tab_spanner(label="Torus", columns=[col for col in df.columns if "Torus" in col])
    .tab_spanner(label="Ellipsoid", columns=[col for col in df.columns if "Ellipsoid" in col])
    .tab_spanner(label="Saddle", columns=[col for col in df.columns if "Saddle" in col])
    .fmt_number(columns=[col for col in df.columns if col not in ["Model", "Metric"]], decimals=3)
)

# Function to apply styles to the best values
def apply_best_value_styles(df, table):
    for metric in df['Metric'].unique():
        metric_df = df[df['Metric'] == metric]
        for col in df.columns[2:]:
            if metric == 'PCC':
                best_value = metric_df[col].max()
            else:  # For RMSE and other metrics where lower is better
                best_value = metric_df[col].min()
                
            best_indices = metric_df[metric_df[col] == best_value].index.tolist()
            for index in best_indices:
                table = table.tab_style(
                    style=style.text(weight="bold"),
                    locations=loc.body(columns=[col], rows=[index])
                )
    return table

# Apply the styles
table = apply_best_value_styles(df, table)

# Display the table (assuming in Jupyter Notebook)
display(table)

In [ ]:
metric_df

# Conclusion

Hey!

In [ ]:
X, ks = ellipsoid(n_points, seed=19323)
X

array([[ 2.00397824, -0.41691384,  0.71437483],
       [ 1.27475224, -0.11175413,  0.90350592],
       [-1.91620722,  0.99866728,  0.58539095],
       ...,
       [-2.97769204, -0.03548212,  0.12042399],
       [ 2.35053893, -0.31803095,  0.60068419],
       [-0.5358212 , -1.86937732, -0.30733797]])